Importing Libraries

In [42]:
import pandas as  pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier

Loading data from the train.csv 

In [5]:
df = pd.read_csv("train.csv",usecols=["Age","Fare","Survived"])
df.dropna(inplace=True)
df.head()

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500


Applying discritizer

In [30]:
x = df.iloc[:,1:]
y = df.iloc[:,:1]

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

Transforming couloms Age and Fare 

In [31]:
kbin_age = KBinsDiscretizer(n_bins=5,encode="ordinal",strategy="kmeans")
kbin_fare = KBinsDiscretizer(n_bins=5,encode="ordinal",strategy="kmeans")
transform = ColumnTransformer([
    ("first",kbin_age,[0]),
    ("second",kbin_fare,[1])
])
x_train_transform = transform.fit_transform(x_train)
x_test_transform = transform.fit_transform(x_test)

Shows the ranges of Age and Fare

In [34]:
transform.named_transformers_["first"].bin_edges_

array([array([ 1.        , 13.54505632, 25.49492263, 36.14664502, 48.33333333,
              62.        ])                                                   ],
      dtype=object)

In [35]:
transform.named_transformers_["second"].bin_edges_

array([array([  0.        ,  35.74420523,  90.82735435, 176.31875   ,
              369.9271    , 512.3292    ])                           ],
      dtype=object)

In [36]:
output = pd.DataFrame({
    'age':x_train['Age'],
    'age_trf':x_train_transform[:,0],
    'fare':x_train['Fare'],
    'fare_trf':x_train_transform[:,1]
})
output['age_labels'] = pd.cut(x=x_train['Age'],
                                    bins=transform.named_transformers_['first'].bin_edges_[0].tolist())
output['fare_labels'] = pd.cut(x=x_train['Fare'],
                                    bins=transform.named_transformers_['second'].bin_edges_[0].tolist())
output.sample(5)

,age,age_trf,fare,fare_trf,age_labels,fare_labels
143,19.0,1.0,6.7500,0.0,"(13.545, 25.495]","(0.0, 35.744]"
842,30.0,2.0,31.0000,0.0,"(25.495, 36.147]","(0.0, 35.744]"
830,15.0,1.0,14.4542,0.0,"(13.545, 25.495]","(0.0, 35.744]"
597,49.0,3.0,0.0000,0.0,"(48.333, 62.0]",NaN
554,22.0,1.0,7.7750,0.0,"(13.545, 25.495]","(0.0, 35.744]"


In [38]:
clf = DecisionTreeClassifier()
clf.fit(x_train_transform,y_train)
y_predict = clf.predict(x_test_transform)
accuracy_score(y_test,y_predict)

0.6293706293706294